In [36]:
import pandas as pd
# pd.set_option('expand_frame_repr', True)
# pd.set_option('max_colwidth', 150)

# nltk.download('twitter_samples')
# nltk.download('punkt')
# nltk.download('wordnet') #lexical db for english language that helps script determine base word 
# nltk.download('averaged_perceptron_tagger') #determines the context of a word in a sentence 
# nltk.download('stopwords')


In [44]:
#import nltk
import re, string, random
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import FreqDist
from nltk import classify
from nltk import NaiveBayesClassifier
from nltk.tokenize import word_tokenize

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)



In [38]:
"""Interface"""

stop_words = stopwords.words('english')

positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')

positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

positive_dataset = [(tweet_dict, "Positive") for tweet_dict in positive_tokens_for_model]
negative_dataset = [(tweet_dict, "Negative") for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]

classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))

Accuracy is: 0.9953333333333333
Most Informative Features
                      :( = True           Negati : Positi =   2069.8 : 1.0
                      :) = True           Positi : Negati =    985.6 : 1.0
                follower = True           Positi : Negati =     34.4 : 1.0
                 welcome = True           Positi : Negati =     31.0 : 1.0
                  arrive = True           Positi : Negati =     30.4 : 1.0
                     bam = True           Positi : Negati =     24.3 : 1.0
                     sad = True           Negati : Positi =     23.8 : 1.0
                followed = True           Negati : Positi =     23.0 : 1.0
                   enjoy = True           Positi : Negati =     21.0 : 1.0
                     x15 = True           Negati : Positi =     18.3 : 1.0
None


In [110]:
#freq distribution test
all_pos_words = get_all_words(negative_cleaned_tokens_list)
freq_dist_pos = FreqDist(all_pos_words)
freq_dist_pos.most_common(10)

[(':(', 4585),
 (':-(', 501),
 ("i'm", 343),
 ('...', 332),
 ('get', 325),
 ('miss', 291),
 ('go', 275),
 ('please', 275),
 ('want', 246),
 ('like', 218)]

In [73]:
#model test
custom_tweet = "I ordered just once from TerribleCo, they screwed up, never used the app again."
custom_tokens = remove_noise(word_tokenize(custom_tweet))

etweet = elonTwts['cleaned'][2]
etoken = remove_noise(etweet)

print(etweet)
print(classifier.classify(dict([token, True] for token in etweet)))

['much', 'appreciate', 'mayor', 'fine']
Positive


In [170]:
# import twint

# c = twint.Config()
# c.Username = "elonmusk"
# c.Search = "tesla"
# c.Limit = 2000
# c.Store_csv = True
# c.Output = "elonmusk_tesla.csv"


# twint.run.Search(c)

In [113]:
import pandas as pd

elonTwts = pd.read_csv("elonmusk_tesla.csv")
nullClms = elonTwts.columns[elonTwts.isna().any()].tolist() #null columns in list 
#elonTwts.columns.values
elonTwts = elonTwts.drop(nullClms,axis=1) #dropping null columns 
#elonTwts.info()

columnsMain = ['user_id','username','date','time','name','tweet','photos','timezone',
               'mentions','replies_count','retweets_count','likes_count','reply_to']
columnsTest = ['user_id','username','date','time','replies_count',
               'retweets_count','likes_count','tweet']

elonTwts = elonTwts[columnsTest]
elonTwts.head()

,user_id,username,date,time,replies_count,retweets_count,likes_count,tweet
0,44196397,elonmusk,2020-05-10,00:23:20,228,118,4119,Exactly
1,44196397,elonmusk,2020-05-09,16:17:07,2457,5205,31262,"I’m not messing around. Absurd & medically irrational behavior in violation of constitutional civil liberties, moreover by *unelected* county offi..."
2,44196397,elonmusk,2020-05-09,15:43:26,196,155,5505,"Much appreciated, Mayor Fine!"
3,44196397,elonmusk,2020-05-09,10:18:14,608,932,10070,Tesla is the biggest manufacturer in California & second biggest exporter
4,44196397,elonmusk,2020-05-09,10:14:28,359,537,4599,Exactly! Tesla knows far more about what needs to be done to be safe through our Tesla China factory experience than an (unelected) interim junior...


In [111]:
['tokenized'] = [word_tokenize(tweet) for tweet in elonTwts['tweet']]
# for tweet in elonTwts['tweet']:
#     tokenized.append(word_tokenize(tweet))
elonTwts['tokenized'] = tokenized

cleaned = []
for tweet in elonTwts['tokenized']:
    cleaned.append(remove_noise(tweet, stop_words))
elonTwts['cleaned'] = cleaned

sentiment = []
for tweet in elonTwts['cleaned']:
    sentiment.append(classifier.classify(dict([token, True] for token in tweet)))
elonTwts['sentiment'] = sentiment    

In [112]:
elonTwts[['tweet','tokenized','cleaned','sentiment']].head()

,tweet,tokenized,cleaned,sentiment
0,Exactly,[Exactly],[exactly],Positive
1,"I’m not messing around. Absurd & medically irrational behavior in violation of constitutional civil liberties, moreover by *unelected* county offi...","[I, ’, m, not, messing, around, ., Absurd, &, medically, irrational, behavior, in, violation, of, constitutional, civil, liberties, ,, moreover, b...","[’, mess, around, absurd, medically, irrational, behavior, violation, constitutional, civil, liberty, moreover, *unelected*, county, official, acc...",Negative
2,"Much appreciated, Mayor Fine!","[Much, appreciated, ,, Mayor, Fine, !]","[much, appreciate, mayor, fine]",Positive
3,Tesla is the biggest manufacturer in California & second biggest exporter,"[Tesla, is, the, biggest, manufacturer, in, California, &, second, biggest, exporter]","[tesla, big, manufacturer, california, second, big, exporter]",Positive
4,Exactly! Tesla knows far more about what needs to be done to be safe through our Tesla China factory experience than an (unelected) interim junior...,"[Exactly, !, Tesla, knows, far, more, about, what, needs, to, be, done, to, be, safe, through, our, Tesla, China, factory, experience, than, an, (...","[exactly, tesla, know, far, need, safe, tesla, china, factory, experience, unelected, interim, junior, official, alameda, county]",Positive


In [107]:
#elonTwts[['tweet','tokenized','cleaned']].loc[1]

In [109]:
elonTwts['sentiment'].value_counts()

Positive    801
Negative    598
Name: sentiment, dtype: int64

In [108]:
elonWords = get_all_words(elonTwts.cleaned)

from nltk import FreqDist

freq_dist_E = FreqDist(elonWords)
freq_dist_E.most_common(10)

[('tesla', 904),
 ('’', 313),
 ('http', 217),
 ('…', 207),
 ('car', 206),
 ('model', 106),
 ('year', 98),
 ('good', 89),
 ('work', 83),
 ('make', 82)]

In [356]:
sent = elonTwts['cleaned'][1396]

In [357]:
classifier.classify(dict([token, True] for token in sent))

'Positive'

In [30]:
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')
tweet_tokens = twitter_samples.tokenized('positive_tweets.json') 

positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
    
positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

In [248]:
print(tweet_tokens[0])

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'being', 'top', 'engaged', 'members', 'in', 'my', 'community', 'this', 'week', ':)']


In [249]:
#normalization
from nltk.tag import pos_tag #https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
from nltk.stem.wordnet import WordNetLemmatizer

print(pos_tag(tweet_tokens[0]))

[('#FollowFriday', 'JJ'), ('@France_Inte', 'NNP'), ('@PKuchly57', 'NNP'), ('@Milipol_Paris', 'NNP'), ('for', 'IN'), ('being', 'VBG'), ('top', 'JJ'), ('engaged', 'VBN'), ('members', 'NNS'), ('in', 'IN'), ('my', 'PRP$'), ('community', 'NN'), ('this', 'DT'), ('week', 'NN'), (':)', 'NN')]


In [252]:
def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

print(lemmatize_sentence(tweet_tokens[0]))

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'be', 'top', 'engage', 'member', 'in', 'my', 'community', 'this', 'week', ':)']


In [18]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

all_pos_words = get_all_words(positive_cleaned_tokens_list)

NameError: name 'positive_cleaned_tokens_list' is not defined

In [317]:
from nltk import FreqDist

freq_dist_pos = FreqDist(all_pos_words)
freq_dist_pos.most_common(10)

[(':)', 3691),
 (':-)', 701),
 (':d', 658),
 ('thanks', 388),
 ('follow', 357),
 ('love', 333),
 ('...', 290),
 ('good', 283),
 ('get', 263),
 ('thank', 253)]

In [20]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

In [31]:
import random

positive_dataset = [(tweet_dict, "Positive") for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative") for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:8000]
test_data = dataset[8000:]

In [32]:
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))


Accuracy is: 0.9975
Most Informative Features
                      :( = True           Negati : Positi =   2369.4 : 1.0
                      :) = True           Positi : Negati =   1126.5 : 1.0
                follower = True           Positi : Negati =     24.0 : 1.0
                     bam = True           Positi : Negati =     22.2 : 1.0
                     x15 = True           Negati : Positi =     21.2 : 1.0
                     sad = True           Negati : Positi =     21.2 : 1.0
               community = True           Positi : Negati =     15.5 : 1.0
                    blog = True           Positi : Negati =     14.2 : 1.0
                    glad = True           Positi : Negati =     14.1 : 1.0
                 welcome = True           Positi : Negati =     14.0 : 1.0
None


In [347]:


custom_tweet = ""

custom_tokens = remove_noise(word_tokenize(custom_tweet))

print(classifier.classify(dict([token, True] for token in custom_tokens)))

Positive


In [38]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords, twitter_samples
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

import re, string, random

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

if __name__ == "__main__":

    positive_tweets = twitter_samples.strings('positive_tweets.json')
    negative_tweets = twitter_samples.strings('negative_tweets.json')
    text = twitter_samples.strings('tweets.20150430-223406.json')
    tweet_tokens = twitter_samples.tokenized('positive_tweets.json')[0]

    stop_words = stopwords.words('english')

    positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
    negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

    positive_cleaned_tokens_list = []
    negative_cleaned_tokens_list = []

    for tokens in positive_tweet_tokens:
        positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    for tokens in negative_tweet_tokens:
        negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    all_pos_words = get_all_words(positive_cleaned_tokens_list)

    freq_dist_pos = FreqDist(all_pos_words)
    print(freq_dist_pos.most_common(10))

    positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
    negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

    positive_dataset = [(tweet_dict, "Positive")
                         for tweet_dict in positive_tokens_for_model]

    negative_dataset = [(tweet_dict, "Negative")
                         for tweet_dict in negative_tokens_for_model]

    dataset = positive_dataset + negative_dataset

    random.shuffle(dataset)

    train_data = dataset[:7000]
    test_data = dataset[7000:]

    classifier = NaiveBayesClassifier.train(train_data)

    print("Accuracy is:", classify.accuracy(classifier, test_data))

    print(classifier.show_most_informative_features(10))

    custom_tweet = "I ordered just once from TerribleCo, they screwed up, never used the app again."

    custom_tokens = remove_noise(word_tokenize(custom_tweet))
    
   # print(custom_tweet, classifier.classify(dict([token, True] for token in custom_tokens)))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]
Accuracy is: 0.9963333333333333
Most Informative Features
                      :( = True           Negati : Positi =   2061.9 : 1.0
                      :) = True           Positi : Negati =   1649.7 : 1.0
                     sad = True           Negati : Positi =     57.6 : 1.0
                follower = True           Positi : Negati =     21.2 : 1.0
                     bam = True           Positi : Negati =     20.1 : 1.0
                     x15 = True           Negati : Positi =     18.5 : 1.0
               community = True           Positi : Negati =     15.5 : 1.0
                 welcome = True           Positi : Negati =     13.7 : 1.0
                    glad = True           Positi : Negati =     12.9 : 1.0
                 perfect = True           Positi : Negati =     12.2 : 1.0
None


Most Informative Features
                      :( = True           Negati : Positi =   2061.9 : 1.0
                      :) = True           Positi : Negati =   1649.7 : 1.0
                     sad = True           Negati : Positi =     57.6 : 1.0
                follower = True           Positi : Negati =     21.2 : 1.0
                     bam = True           Positi : Negati =     20.1 : 1.0
                     x15 = True           Negati : Positi =     18.5 : 1.0
               community = True           Positi : Negati =     15.5 : 1.0
                 welcome = True           Positi : Negati =     13.7 : 1.0
                    glad = True           Positi : Negati =     12.9 : 1.0
                 perfect = True           Positi : Negati =     12.2 : 1.0
                     ugh = True           Negati : Positi =     11.1 : 1.0
               goodnight = True           Positi : Negati =     10.9 : 1.0
                     bro = True           Positi : Negati =     10.9 : 1.0